In [9]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [10]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

// TODO(Johann) could these be moved to json configuration files?
val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val IMPORT_CONFIGURATION_FILE = "../../data/provision/import-configuration.json"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-13.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=51c46d61-4fd6-4a20-b54c-03c5e6f122bc, username=Johann-13.1)

In [11]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=68ab5805-98c9-4b07-9d20-bf6b560875e5, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=192cf94f-1e7e-4840-8514-24309cee7371, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=a1b4b536-34a3-468f-96d8-63c1800a7f53, name=ING RON, unit=Currency(value=RON))
AccountTO(id=e8c7469d-01aa-4509-8dc6-81755504ae06, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=d985d5ec-1868-4de3-908a-0e119bdcc686, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=712df92e-3b5d-4bc7-83a1-8cbf04a8a492, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=6b906673-edd7-41be-ab10-09f3184cbf8e, name=Other, unit=Currency(value=RON))
AccountTO(id=12888425-808e-444f-acb1-eab574a03733, name=BT RON, unit=Currency(value=RON))
AccountTO(id=bff642a8-7731-49a4-bdf5-64b1dab78d29, name=BT Economy RON, unit=Currency(value=RON))

In [12]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=3ce10b59-0db6-4c89-a0d5-58ddae7fe1ed, name=Expenses)
FundTO(id=574202a3-5d26-4e79-bb7b-b2c683d25604, name=Work Income)
FundTO(id=b6799394-d991-4ff7-a98b-228da979b6ec, name=Savings)
FundTO(id=1f280e72-21a2-44da-91de-6995fd4a0f66, name=Gifts)

In [13]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawJson: String = File(IMPORT_CONFIGURATION_FILE).readText()
val jsonFormat = Json { ignoreUnknownKeys = true }
val importConfiguration = jsonFormat.decodeFromString<ImportConfigurationTO>(dataConfigurationRawJson)

In [14]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=6a91e5f0-187c-4993-aaf0-15445e5ba289, status=COMPLETED, reason=null)